In [1]:

import sqlite3
import psycopg
import pandas as pd
import json
import random
from datetime import datetime, timedelta

In [2]:
!pwd

/home/student20/laboratorium5


In [3]:
with open("/home/student20/laboratorium5/database_creds.json") as db_con_file:
   creds = json.loads(db_con_file.read())

In [4]:
user = creds['user_name']
password = creds['password']
host = creds['host_name']
port = creds['port_number']
db_name = creds['db_name']
connection_string = "postgresql+psycopg://{user}:{password}@{host}:{port}/{db_name}".format(user=user, password=password, host=host, port=port, db_name=db_name)


In [5]:
class SklepZBronia:
    def __init__(self, creds):
        self.creds = creds
        self.sqlite_conn = None
        self.pg_conn = None
        self.setup_connections()
    
    def setup_connections(self):
        # SQLite
        self.sqlite_conn = sqlite3.connect('sklep_bron.db')
        self.sqlite_conn.row_factory = sqlite3.Row
        
        # PostgreSQL
        try:
            self.pg_conn = psycopg.connect(
                dbname=self.creds['db_name'],
                user=self.creds['user_name'],
                password=self.creds['password'],
                host=self.creds['host_name'],
                port=self.creds['port_number']
            )
        except psycopg.Error as e:
            print(f"Błąd połączenia z PostgreSQL: {e}")
            self.pg_conn = None # Ensure pg_conn is None if connection fails
    
    def create_tables(self):
        # SQLite
        with self.sqlite_conn:
            self.sqlite_conn.execute('''
                CREATE TABLE IF NOT EXISTS kategorie (
                    id INTEGER PRIMARY KEY,
                    nazwa TEXT NOT NULL,
                    opis TEXT
                )
            ''')
            
            self.sqlite_conn.execute('''
                CREATE TABLE IF NOT EXISTS produkty (
                    id INTEGER PRIMARY KEY,
                    nazwa TEXT NOT NULL,
                    opis TEXT,
                    cena REAL NOT NULL,
                    stan_magazynowy INTEGER NOT NULL,
                    kategoria_id INTEGER,
                    FOREIGN KEY (kategoria_id) REFERENCES kategorie(id)
                )
            ''')
        
        # PostgreSQL
        if self.pg_conn:
            try:
                with self.pg_conn.cursor() as cur:
                    cur.execute('''
                        CREATE TABLE IF NOT EXISTS kategorie (
                            id SERIAL PRIMARY KEY,
                            nazwa VARCHAR(100) NOT NULL,
                            opis TEXT
                        )
                    ''')
                    
                    cur.execute('''
                        CREATE TABLE IF NOT EXISTS produkty (
                            id SERIAL PRIMARY KEY,
                            nazwa VARCHAR(100) NOT NULL,
                            opis TEXT,
                            cena DECIMAL(10,2) NOT NULL,
                            stan_magazynowy INTEGER NOT NULL,
                            kategoria_id INTEGER,
                            FOREIGN KEY (kategoria_id) REFERENCES kategorie(id)
                        )
                    ''')
                self.pg_conn.commit()
            except psycopg.Error as e:
                print(f"Error creating PostgreSQL tables: {e}")
                if self.pg_conn and not self.pg_conn.closed:
                    self.pg_conn.rollback()

    def generate_test_data(self, count=200):
        kategorie_data = [
            ("Broń krótka", "Pistolety i rewolwery"),
            ("Broń długa", "Karabiny i strzelby"),
            ("Akcesoria", "Akcesoria strzeleckie")
        ]
        
        kategorie_for_export = []
        for i, (nazwa, opis) in enumerate(kategorie_data):
            kategorie_for_export.append((i + 1, nazwa, opis))

        produkty = []
        for i in range(count):
            kategoria_id_for_export = random.randint(1, len(kategorie_data))
            produkt = {
                'id': i+1,
                'nazwa': f"Produkt {i+1}",
                'opis': f"Opis produktu {i+1}",
                'cena': round(random.uniform(100.0, 10000.0), 2),
                'stan_magazynowy': random.randint(0, 100),
                'kategoria_id': kategoria_id_for_export 
            }
            produkty.append(produkt)
        
        return kategorie_for_export, produkty
    
    def export_to_json(self, kategorie, produkty):
        kategorie_json = [{'id': k[0], 'nazwa': k[1], 'opis': k[2]} for k in kategorie]
        data = {
            'kategorie': kategorie_json,
            'produkty': produkty
        }
        
        with open('dane_testowe.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
    
    def export_to_csv(self, kategorie, produkty):
        df_kategorie = pd.DataFrame(kategorie, columns=['id', 'nazwa', 'opis'])
        df_kategorie.to_csv('kategorie.csv', index=False, encoding='utf-8')
        
        df_produkty = pd.DataFrame(produkty)
        df_produkty.to_csv('produkty.csv', index=False, encoding='utf-8')
    
    def import_from_json(self, filename):
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            kategorie_sqlite = [(k['id'], k['nazwa'], k['opis']) for k in data['kategorie']]
            produkty_sqlite = [(p['id'], p['nazwa'], p['opis'], p['cena'], p['stan_magazynowy'], p['kategoria_id']) for p in data['produkty']]

            with self.sqlite_conn:
                self.sqlite_conn.executemany(
                    'INSERT INTO kategorie (id, nazwa, opis) VALUES (?, ?, ?)',
                    kategorie_sqlite
                )
                self.sqlite_conn.executemany(
                    'INSERT INTO produkty (id, nazwa, opis, cena, stan_magazynowy, kategoria_id) VALUES (?, ?, ?, ?, ?, ?)',
                    produkty_sqlite
                )
                self.sqlite_conn.commit()
        except sqlite3.Error as e:
            print(f"SQLite error during JSON import: {e}")
            self.sqlite_conn.rollback() # Rollback on error
        except Exception as e:
            print(f"Generic error during JSON import: {e}")


    def import_from_csv(self):
        if self.pg_conn and not self.pg_conn.closed:
            try:
                with self.pg_conn.cursor() as cur:
                    # Import kategorie
                    # The COPY command with HEADER TRUE tells PostgreSQL to handle the header row.
                    # We open the file in binary mode ('rb') because copy.write() expects bytes.
                    copy_sql_kategorie = "COPY kategorie (id, nazwa, opis) FROM STDIN (FORMAT CSV, HEADER TRUE, NULL '', DELIMITER ',')"
                    with open('kategorie.csv', 'rb') as f_kategorie:
                        with cur.copy(copy_sql_kategorie) as copy_op_k:
                            while chunk := f_kategorie.read(8192): # Read file in chunks
                                copy_op_k.write(chunk)
                    
                    # Import produkty
                    copy_sql_produkty = "COPY produkty (id, nazwa, opis, cena, stan_magazynowy, kategoria_id) FROM STDIN (FORMAT CSV, HEADER TRUE, NULL '', DELIMITER ',')"
                    with open('produkty.csv', 'rb') as f_produkty:
                        with cur.copy(copy_sql_produkty) as copy_op_p:
                            while chunk := f_produkty.read(8192):
                                copy_op_p.write(chunk)
                
                self.pg_conn.commit()
            except psycopg.Error as e:
                print(f"PostgreSQL Error during CSV import: {e}")
                if self.pg_conn and not self.pg_conn.closed:
                    self.pg_conn.rollback() # Rollback on error
            except FileNotFoundError as e:
                print(f"CSV file not found during import: {e}")
                if self.pg_conn and not self.pg_conn.closed:
                    self.pg_conn.rollback()
            except Exception as e:
                print(f"Generic error during CSV import: {e}")
                if self.pg_conn and not self.pg_conn.closed:
                    self.pg_conn.rollback()
        else:
            print("Skipping PostgreSQL CSV import (no valid connection).")

    def backup_database(self):
        try:
            with sqlite3.connect('sklep_bron.db') as src_conn: # Ensure it uses the instance's DB name
                with open('backup_sklep.sql', 'w', encoding='utf-8') as f:
                    for line in src_conn.iterdump():
                        f.write(f'{line}\n')
            print("SQLite backup successful to backup_sklep.sql")
        except sqlite3.Error as e:
            print(f"Error backing up SQLite database: {e}")
        except Exception as e:
            print(f"Generic error during SQLite backup: {e}")

    def clear_database(self):
        # SQLite
        try:
            with self.sqlite_conn:
                self.sqlite_conn.execute('DELETE FROM produkty')
                self.sqlite_conn.execute('DELETE FROM kategorie')
                self.sqlite_conn.commit()
        except sqlite3.Error as e:
            print(f"Error clearing SQLite database: {e}")
            self.sqlite_conn.rollback()
        
        # PostgreSQL
        if self.pg_conn and not self.pg_conn.closed:
            try:
                with self.pg_conn.cursor() as cur:
                    # Using TRUNCATE is faster and also resets sequences if RESTART IDENTITY is used.
                    cur.execute('TRUNCATE TABLE produkty, kategorie RESTART IDENTITY CASCADE')
                self.pg_conn.commit()
            except psycopg.Error as e:
                print(f"Error clearing PostgreSQL database: {e}")
                if self.pg_conn and not self.pg_conn.closed:
                    self.pg_conn.rollback()

    def close_connections(self):
        if self.sqlite_conn:
            try:
                self.sqlite_conn.close()
            except Exception as e:
                print(f"Error closing SQLite connection: {e}")
        if self.pg_conn and not self.pg_conn.closed:
            try:
                self.pg_conn.close()
            except Exception as e:
                print(f"Error closing PostgreSQL connection: {e}")

    
    def add_sample_data(self):
        """
        Dodaje przykładowe dane do obu baz danych
        """
        # Dane kategorii
        sample_categories = [
            (1, "Broń krótka", "Pistolety i rewolwery"),
            (2, "Broń długa", "Karabiny i strzelby"),
            (3, "Akcesoria", "Akcesoria strzeleckie")
        ]
    
        # Dane produktów
        sample_products = [
            (1, "Pistolet Glock 19", "9mm pistolet samopowtarzalny", 2999.99, 15, 1),
            (2, "Karabin Sako 85", "Karabin myśliwski kaliber .308", 9999.99, 8, 2),
            (3, "Celownik optyczny", "Celownik 3-9x40", 1299.99, 25, 3),
            (4, "Rewolwer Smith & Wesson", "Revolver .357 Magnum", 3999.99, 12, 1),
            (5, "Strzelba Remington", "Strzelba pump-action 12 kaliber", 4999.99, 18, 2),
            (6, "Torba taktyczna", "Torba na sprzęt strzelecki", 599.99, 30, 3)
        ]
    
        try:
            # SQLite
            with self.sqlite_conn:
                self.sqlite_conn.executemany(
                    'INSERT OR REPLACE INTO kategorie (id, nazwa, opis) VALUES (?, ?, ?)',
                    sample_categories
                )
                self.sqlite_conn.executemany(
                    'INSERT OR REPLACE INTO produkty (id, nazwa, opis, cena, stan_magazynowy, kategoria_id) VALUES (?, ?, ?, ?, ?, ?)',
                    sample_products
                )
    
            # PostgreSQL
            if self.pg_conn and not self.pg_conn.closed:
                with self.pg_conn.cursor() as cur:
                    cur.executemany(
                        'INSERT INTO kategorie (id, nazwa, opis) VALUES (%s, %s, %s) ON CONFLICT (id) DO UPDATE SET nazwa = EXCLUDED.nazwa, opis = EXCLUDED.opis',
                        sample_categories
                    )
                    cur.executemany(
                        'INSERT INTO produkty (id, nazwa, opis, cena, stan_magazynowy, kategoria_id) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO UPDATE SET nazwa = EXCLUDED.nazwa, opis = EXCLUDED.opis, cena = EXCLUDED.cena, stan_magazynowy = EXCLUDED.stan_magazynowy, kategoria_id = EXCLUDED.kategoria_id',
                        sample_products
                    )
                self.pg_conn.commit()
    
            print("Przykładowe dane zostały dodane do obu baz danych.")
    
        except sqlite3.Error as e:
            print(f"Błąd SQLite podczas dodawania danych: {e}")
            if self.sqlite_conn:
                self.sqlite_conn.rollback()
        except psycopg.Error as e:
            print(f"Błąd PostgreSQL podczas dodawania danych: {e}")
            if self.pg_conn and not self.pg_conn.closed:
                self.pg_conn.rollback()

    
    def print_all_tables(self):
        """
        Wyświetla zawartość wszystkich tabel w obu bazach danych
        """
        try:
            # SQLite
            print("\n=== Zawartość tabel SQLite ===")
            with self.sqlite_conn:
                # Pobierz listę wszystkich tabel
                cursor = self.sqlite_conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
                tables = cursor.fetchall()
                
                for table_name in tables:
                    table = table_name[0]
                    print(f"\nTabela: {table}")
                    print("-" * 50)
                    
                    # Pobierz kolumny
                    cursor.execute(f"PRAGMA table_info({table})")
                    columns = [col[1] for col in cursor.fetchall()]
                    print(" | ".join(columns))
                    print("-" * 50)
                    
                    # Pobierz dane
                    cursor.execute(f"SELECT * FROM {table}")
                    rows = cursor.fetchall()
                    for row in rows:
                        print(" | ".join(str(value) for value in row))
                    
                    print("\nLiczba wierszy:", len(rows))
                    print("-" * 50)
    
            # PostgreSQL
            if self.pg_conn and not self.pg_conn.closed:
                print("\n=== Zawartość tabel PostgreSQL ===")
                with self.pg_conn.cursor() as cur:
                    # Pobierz listę wszystkich tabel
                    cur.execute("""
                        SELECT tablename 
                        FROM pg_tables 
                        WHERE schemaname = 'public'
                    """)
                    tables = cur.fetchall()
                    
                    for table_name in tables:
                        table = table_name[0]
                        print(f"\nTabela: {table}")
                        print("-" * 50)
                        
                        # Pobierz dane
                        cur.execute(f"SELECT * FROM {table}")
                        rows = cur.fetchall()
                        
                        # Pobierz nazwy kolumn
                        columns = [desc[0] for desc in cur.description]
                        print(" | ".join(columns))
                        print("-" * 50)
                        
                        for row in rows:
                            print(" | ".join(str(value) for value in row))
                        
                        print("\nLiczba wierszy:", len(rows))
                        print("-" * 50)
    
        except sqlite3.Error as e:
            print(f"Błąd SQLite podczas wyświetlania tabel: {e}")
        except psycopg.Error as e:
            print(f"Błąd PostgreSQL podczas wyświetlania tabel: {e}")


In [6]:


if __name__ == "__main__":
    creds = {
        'user_name': user, 
        'password': password,
        'host_name': host, 
        'port_number': port, 
        'db_name': db_name,
    }

    sklep = SklepZBronia(creds)

    # Clear database (optional, for clean testing)
    # print("Clearing database...")
    # sklep.clear_database()

    # Tworzenie tabel
    print("Creating tables...")
    sklep.create_tables()

    # Generowanie i eksport danych
    print("Generating and exporting data...")
    # Note: generate_test_data gives kategorie with IDs 1,2,3
    kategorie, produkty = sklep.generate_test_data(count=50) # Reduced count for faster testing
    sklep.export_to_json(kategorie, produkty)
    sklep.export_to_csv(kategorie, produkty)

    # Import danych
    # Clear tables before import if you run this multiple times to avoid PK conflicts
    # sklep.clear_database() # Re-create tables if clearing also drops them
    # sklep.create_tables()  # Or ensure clear_database only deletes data

    print("Importing from JSON (SQLite)...")
    try:
        sklep.import_from_json('dane_testowe.json')
    except sqlite3.IntegrityError as e:
        print(f"SQLite IntegrityError (likely data already exists): {e}")
        print("Skipping SQLite import to avoid duplicate data.")
    
    print("Importing from CSV (PostgreSQL)...")
    if sklep.pg_conn:
        try:
            # To avoid issues with re-running, clear PG tables before import
            with sklep.pg_conn.cursor() as cur:
                cur.execute("TRUNCATE TABLE produkty, kategorie RESTART IDENTITY CASCADE")
            sklep.pg_conn.commit()
            sklep.import_from_csv()
        except psycopg.Error as e:
            print(f"PostgreSQL Error during CSV import: {e}")
            # You might want to rollback if part of a transaction failed
            if sklep.pg_conn and not sklep.pg_conn.closed:
                 sklep.pg_conn.rollback()
    else:
        print("Skipping PostgreSQL import (no connection).")

    print("Backing up SQLite database...")
    sklep.backup_database()
    
    print("Operations complete.")
    

Creating tables...
Generating and exporting data...
Importing from JSON (SQLite)...
SQLite error during JSON import: UNIQUE constraint failed: kategorie.id
Importing from CSV (PostgreSQL)...
Backing up SQLite database...
SQLite backup successful to backup_sklep.sql
Operations complete.


In [7]:
# Dodanie przykładowych danych
print("Dodawanie przykładowych danych...")
sklep.add_sample_data()

Dodawanie przykładowych danych...
Przykładowe dane zostały dodane do obu baz danych.


In [8]:
# Wyświetl zawartość wszystkich tabel
print("Wyświetlam zawartość wszystkich tabel...")
sklep.print_all_tables()

Wyświetlam zawartość wszystkich tabel...

=== Zawartość tabel SQLite ===

Tabela: produkty
--------------------------------------------------
id | nazwa | opis | cena | stan_magazynowy | kategoria_id
--------------------------------------------------
1 | Pistolet Glock 19 | 9mm pistolet samopowtarzalny | 2999.99 | 15 | 1
2 | Karabin Sako 85 | Karabin myśliwski kaliber .308 | 9999.99 | 8 | 2
3 | Celownik optyczny | Celownik 3-9x40 | 1299.99 | 25 | 3
4 | Rewolwer Smith & Wesson | Revolver .357 Magnum | 3999.99 | 12 | 1
5 | Strzelba Remington | Strzelba pump-action 12 kaliber | 4999.99 | 18 | 2
6 | Torba taktyczna | Torba na sprzęt strzelecki | 599.99 | 30 | 3
7 | Produkt 7 | Opis produktu 7 | 9081.19 | 9 | 2
8 | Produkt 8 | Opis produktu 8 | 8267.38 | 11 | 2
9 | Produkt 9 | Opis produktu 9 | 3752.2 | 99 | 2
10 | Produkt 10 | Opis produktu 10 | 9704.96 | 56 | 1
11 | Produkt 11 | Opis produktu 11 | 1121.78 | 20 | 3
12 | Produkt 12 | Opis produktu 12 | 8164.16 | 81 | 2
13 | Produkt 13 | Opi

In [9]:
sklep.close_connections()